In [3]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
import seaborn as sns 
import os 
import sys

In [106]:
"""loading data in"""
df = pd.read_csv('../data/train.csv')
df.head()

,TRAN_AMT,ACCT_PRE_TRAN_AVAIL_BAL,CUST_AGE,OPEN_ACCT_CT,WF_dvc_age,PWD_UPDT_TS,CARR_NAME,RGN_NAME,STATE_PRVNC_TXT,ALERT_TRGR_CD,...,CUST_STATE,PH_NUM_UPDT_TS,CUST_SINCE_DT,TRAN_TS,TRAN_DT,ACTN_CD,ACTN_INTNL_TXT,TRAN_TYPE_CD,ACTVY_DT,FRAUD_NONFRAUD
0,5.38,23619.91,47,4,2777,1/16/2018 11:3:58,cox communications inc.,southwest,nevada,MOBL,...,NV,2/24/2021 15:55:10,1993-01-06 00:00:00,5/3/2021 18:3:58,5/3/2021,SCHPMT,P2P_COMMIT,P2P,5/3/2021,Non-Fraud
1,65.19,0.00,45,5,2721,NaN,charter communications,southwest,california,MOBL,...,CA,NaN,1971-01-07 00:00:00,1/13/2021 19:19:37,1/13/2021,SCHPMT,P2P_COMMIT,P2P,1/13/2021,Non-Fraud
2,54.84,34570.63,36,8,1531,12/22/2021 10:42:51,utah broadband llc,mountain,utah,ONLN,...,MD,5/5/2019 1:8:39,1994-02-01 00:00:00,4/8/2021 9:42:51,4/8/2021,SCHPMT,P2P_COMMIT,P2P,4/8/2021,Fraud
3,0.01,0.00,62,3,835,2/8/2020 7:28:31,t-mobile usa inc.,southwest,california,MOBL,...,NV,2/16/2019 6:45:37,2001-11-01 00:00:00,8/10/2021 15:28:31,8/10/2021,SCHPMT,P2P_COMMIT,P2P,8/10/2021,Non-Fraud
4,497.08,12725.18,81,2,1095,12/28/2020 12:12:44,cogent communications,south central,texas,MOBL,...,UT,5/8/2020 10:27:6,1987-02-07 00:00:00,6/27/2021 11:12:44,6/27/2021,SCHPMT,P2P_COMMIT,P2P,6/27/2021,Fraud


In [107]:
""" Understanding the data """
# df=df.dropna()
df.groupby('FRAUD_NONFRAUD').count()
#Oh boy the classes are imballanced but its not too bad! 
#I'll try to use basic methods first before I try evening out the data with synthetic data
#(I'm not sure how well that would work given the data is synthetically generated already)

,TRAN_AMT,ACCT_PRE_TRAN_AVAIL_BAL,CUST_AGE,OPEN_ACCT_CT,WF_dvc_age,PWD_UPDT_TS,CARR_NAME,RGN_NAME,STATE_PRVNC_TXT,ALERT_TRGR_CD,...,CUST_ZIP,CUST_STATE,PH_NUM_UPDT_TS,CUST_SINCE_DT,TRAN_TS,TRAN_DT,ACTN_CD,ACTN_INTNL_TXT,TRAN_TYPE_CD,ACTVY_DT
FRAUD_NONFRAUD,,,,,,,,,,,,,,,,,,,,,
Fraud,4164,4164,4164,4164,4164,3628,2195,2195,2195,4164,...,4164,4164,2816,4164,4164,4164,4164,4164,4164,4164
Non-Fraud,9836,9836,9836,9836,9836,7247,9096,9096,9096,9836,...,9836,9800,4123,9836,9836,9836,9836,9836,9836,9836


In [108]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
df['label'] = le.fit_transform(df['FRAUD_NONFRAUD'])
df.groupby('label').count()


,TRAN_AMT,ACCT_PRE_TRAN_AVAIL_BAL,CUST_AGE,OPEN_ACCT_CT,WF_dvc_age,PWD_UPDT_TS,CARR_NAME,RGN_NAME,STATE_PRVNC_TXT,ALERT_TRGR_CD,...,CUST_STATE,PH_NUM_UPDT_TS,CUST_SINCE_DT,TRAN_TS,TRAN_DT,ACTN_CD,ACTN_INTNL_TXT,TRAN_TYPE_CD,ACTVY_DT,FRAUD_NONFRAUD
label,,,,,,,,,,,,,,,,,,,,,
0,4164,4164,4164,4164,4164,3628,2195,2195,2195,4164,...,4164,2816,4164,4164,4164,4164,4164,4164,4164,4164
1,9836,9836,9836,9836,9836,7247,9096,9096,9096,9836,...,9800,4123,9836,9836,9836,9836,9836,9836,9836,9836


In [109]:
""" I wanna drop inconsistent columns, label encode columns that I wanna keep + scale it for non-tree algos
drop columns: TRAN_TYPE_CD, ACTN_INTNL_TXT, ACTN_CD, DVC_TYPE_TXT, STATE_PRVNC_TXT, RGN_NAME
CARR_NAME, PWD_UPDT_TS, PH_NUM_UPDT_TS
"""
# dropping 
drop = ['TRAN_TYPE_CD', 'ACTN_INTNL_TXT', 'ACTN_CD', 'DVC_TYPE_TXT', 'STATE_PRVNC_TXT',
       'RGN_NAME', 'CARR_NAME', 'PWD_UPDT_TS', 'PH_NUM_UPDT_TS', 'FRAUD_NONFRAUD']
df = df.drop(drop, axis=1)
# drop dates for now need to convert them later! I think they could be useful
drop_dates = ['ACTVY_DT','TRAN_DT','TRAN_TS', 'CUST_SINCE_DT', 'CUST_SINCE_DT']
df = df.drop(drop_dates, axis=1)
# label encodeing 
df['CUST_STATE'] = le.fit_transform(df['CUST_STATE'])
df['AUTHC_PRIM_TYPE_CD'] = le.fit_transform(df['AUTHC_PRIM_TYPE_CD'])
df['ALERT_TRGR_CD'] = le.fit_transform(df['ALERT_TRGR_CD'])
df['AUTHC_SCNDRY_STAT_TXT'] = le.fit_transform(df['AUTHC_SCNDRY_STAT_TXT'])
df['ALERT_TRGR_CD'] = le.fit_transform(df['ALERT_TRGR_CD'])
df.dropna()
df.groupby('label').count()

,TRAN_AMT,ACCT_PRE_TRAN_AVAIL_BAL,CUST_AGE,OPEN_ACCT_CT,WF_dvc_age,ALERT_TRGR_CD,AUTHC_PRIM_TYPE_CD,AUTHC_SCNDRY_STAT_TXT,CUST_ZIP,CUST_STATE
label,,,,,,,,,,
0,4164,4164,4164,4164,4164,4164,4164,4164,4164,4164
1,9836,9836,9836,9836,9836,9836,9836,9836,9836,9836


In [114]:
"""Now we do machine learning... try basic stuff first trees and isolation forest
Work on this later!!!!"""
from sklearn.ensemble import IsolationForest
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix

params = {} # will be useful if i hyper paramiter tune ya know
def iso_forest(X_train, X_test, y_train, y_test, params):
    clf = IsolationForest()
    print('Fitting...')
    clf.fit(X_train, y_train)
    print('Done fitting...')
    pred = clf.predict(X_test)
    print(confusion_matrix(y_test, pred))
    print(classification_report(y_test, pred))
    

keep = ['TRAN_AMT', 'ACCT_PRE_TRAN_AVAIL_BAL', 'CUST_AGE', 'OPEN_ACCT_CT', 'WF_dvc_age', 'ALERT_TRGR_CD', 'AUTHC_PRIM_TYPE_CD', 'AUTHC_SCNDRY_STAT_TXT', 'CUST_ZIP', 'CUST_STATE']

X = df[keep]
y = df['label']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)
# print(y_test.unique())
# iso_forest(X_train, X_test, y_train, y_test, params)




(9800, 10) (4200, 10) (9800,) (4200,)


In [117]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
def random_forest_gridsearch(X_train, X_test, y_train, y_test):
    param_grid = {
        'max_depth': list(range(3,10)),
        'min_samples_leaf': [5,10,25,40,50],
        'n_estimators': [100,200,300,500,1000]
    }
    print('grid searhing ...')
    grid = GridSearchCV(RandomForestClassifier(), param_grid, cv=5, verbose=True)
    grid.fit(X_train, y_train)
    print(grid.best_params_)
    return grid.best_params_
    
    
def random_forest(X_train, X_test, y_train, y_test, params):
    rf = RandomForestClassifier(**params)
    print('Fitting...')
    rf.fit(X_train, y_train)
    print('Done fitting...')
    pred = rf.predict(X_test)
    print(confusion_matrix(y_test, pred))
    print(classification_report(y_test, pred))
    
params = random_forest_gridsearch(X_train, X_test, y_train, y_test)
random_forest(X_train, X_test, y_train, y_test, params)

grid searhing ...
{'max_depth': 9, 'min_samples_leaf': 5, 'n_estimators': 500}
Fitting...
Done fitting...
[[1065  172]
 [  72 2891]]
              precision    recall  f1-score   support

           0       0.94      0.86      0.90      1237
           1       0.94      0.98      0.96      2963

    accuracy                           0.94      4200
   macro avg       0.94      0.92      0.93      4200
weighted avg       0.94      0.94      0.94      4200



In [122]:
#! conda install -c aterrel xgboost=0.4.0
import xgboost as xgb
def random_forest_gridsearch(X_train, X_test, y_train, y_test):
    param_grid = {
        'max_depth': list(range(3,10)),
        'min_samples_leaf': [5,10,25,40,50],
        'n_estimators': [100,200,300,500,1000]
    }
    print('grid searhing ...')
    grid = GridSearchCV(RandomForestClassifier(), param_grid, cv=5, verbose=True)
    grid.fit(X_train, y_train)
    print(grid.best_params_)
    return grid.best_params_
    
    
def random_forest(X_train, X_test, y_train, y_test, params):
    rf = RandomForestClassifier(**params)
    print('Fitting...')
    rf.fit(X_train, y_train)
    print('Done fitting...')
    pred = rf.predict(X_test)
    print(confusion_matrix(y_test, pred))
    print(classification_report(y_test, pred))

XGBoostError: XGBoost Library (libxgboost.dylib) could not be loaded.
Likely causes:
  * OpenMP runtime is not installed (vcomp140.dll or libgomp-1.dll for Windows, libomp.dylib for Mac OSX, libgomp.so for Linux and other UNIX-like OSes). Mac OSX users: Run `brew install libomp` to install OpenMP runtime.
  * You are running 32-bit Python on a 64-bit OS
Error message(s): ['dlopen(/Users/benjaminpasternak/opt/anaconda3/lib/python3.8/site-packages/xgboost/lib/libxgboost.dylib, 6): Library not loaded: /usr/local/opt/libomp/lib/libomp.dylib\n  Referenced from: /Users/benjaminpasternak/opt/anaconda3/lib/python3.8/site-packages/xgboost/lib/libxgboost.dylib\n  Reason: image not found']
